# Importing All Libraries

In [1]:
import sys
import csv
import random
import math
from datetime import datetime
import multiprocessing
import concurrent.futures
import matplotlib.pyplot as plt
sys.setrecursionlimit(1000000000)

# Generating Datasets

In [2]:
def generate_data(size,filename):
  try:
    data = [int(i) for i in range(1,size+1)]
    random.shuffle(data)
    f = open('Datasets/'+filename,'w')
    for ele in data:
      f.write(str(ele))
      f.write('\n')
  except:
    print('Error Generating ',filename)

In [3]:
print('Generating Data Begin')
start = datetime.now()
with concurrent.futures.ProcessPoolExecutor() as executor:
  executor.submit(generate_data,100000,'HundredThousand.csv')
  executor.submit(generate_data,1000000,'OneMillion.csv')
  executor.submit(generate_data,10000000,'TenMillion.csv')
  executor.submit(generate_data,50000000,'FiftyMillion.csv')
  executor.submit(generate_data,100000000,'HundredMillion.csv')
end = datetime.now()
print('Generating Data Done')
time = end - start
print('Time Elapsed (Seconds) To Generate Data : ',time.total_seconds())

Generating Data Begin
Generating Data Done
Time Elapsed (Seconds) To Generate Data :  171.467131


# Importing Datasets

In [4]:
def read_data(filename):
    try:
        with open('Datasets/'+filename, newline='\n') as f:
            reader = csv.reader(f)
            data = list(reader)
            data = [int(i[0]) for i in data]
    except:
        data = []
    return data

In [5]:
print('Importing Datasets')
start = datetime.now()
with concurrent.futures.ProcessPoolExecutor() as executor:
  ht = executor.submit(read_data,'HundredThousand.csv')
  om = executor.submit(read_data,'OneMillion.csv')
  tm = executor.submit(read_data,'TenMillion.csv')
  fm = executor.submit(read_data,'FiftyMillion.csv')
  hm = executor.submit(read_data,'HundredMillion.csv')
  HundredThousand = ht.result()
  OneMillion = om.result()
  TenMillion = tm.result()
  FiftyMillion = fm.result()
  HundredMillion = hm.result()
end = datetime.now()
time = end - start
print('Time Elapsed (Seconds) To Import Data : ',time.total_seconds())

Importing Datasets
Time Elapsed (Seconds) To Import Data :  94.089241


# Quick Sort & Merge Sort Code

In [6]:
'''
Quick Sort
'''
def sub_partition(array, start, end, idx_pivot):
    'returns the position where the pivot winds up'
    if not (start <= idx_pivot <= end):
        raise ValueError('idx pivot must be between start and end')
    array[start], array[idx_pivot] = array[idx_pivot], array[start]
    pivot = array[start]
    i = start + 1
    j = start + 1
    while j <= end:
        if array[j] <= pivot:
            array[j], array[i] = array[i], array[j]
            i += 1
        j += 1
    array[start], array[i - 1] = array[i - 1], array[start]
    return i - 1

def quickSort(array, process, thread, start=0, end=None):
    if end is None:
        end = len(array) - 1
    if end - start < 1:
        return
    idx_pivot = random.randint(start, end)
    i = sub_partition(array, start, end, idx_pivot)
    #print array, i, idx_pivot
    if process == True and thread==True:
      with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.submit(quickSort,array,False,True, start, i - 1)
        executor.submit(quickSort,array,False,True, i + 1, end)
    elif process == False and thread==True:
      with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.submit(quickSort,array,False,False, start, i - 1)
        executor.submit(quickSort,array,False,False, i + 1, end)
    elif process==False and thread==False:
      quickSort(array,False,False, start, i - 1)
      quickSort(array,False,False, i + 1, end)  

In [7]:
'''
Sequential Merge Sort
'''
def mergeSort(myList):
    if len(myList) > 1:
        mid = len(myList) // 2
        left = myList[:mid]
        right = myList[mid:]
        mergeSort(left)
        mergeSort(right)
        # Two iterators for traversing the two halves
        i = 0
        j = 0
        # Iterator for the main list
        k = 0
        while i < len(left) and j < len(right):
            if left[i] <= right[j]:
              # The value from the left half has been used
              myList[k] = left[i]
              # Move the iterator forward
              i += 1
            else:
                myList[k] = right[j]
                j += 1
            # Move to the next slot
            k += 1
        # For all the remaining values
        while i < len(left):
            myList[k] = left[i]
            i += 1
            k += 1
        while j < len(right):
            myList[k]=right[j]
            j += 1
            k += 1

In [8]:
'''
Parallel Merge Sort
'''
def merge(*args):
    left, right = args[0] if len(args) == 1 else args
    left_length, right_length = len(left), len(right)
    left_index, right_index = 0, 0
    merged = []
    while left_index < left_length and right_index < right_length:
        if left[left_index] <= right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1
    if left_index == left_length:
        merged.extend(right[right_index:])
    else:
        merged.extend(left[left_index:])
    return merged

def merge_sort(data):
    length = len(data)
    if length <= 1:
        return data
    middle = length // 2
    left = merge_sort(data[:middle])
    right = merge_sort(data[middle:])
    return merge(left, right)

def merge_sort_parallel(data):
    processes = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(processes=processes)
    size = int(math.ceil(float(len(data)) / processes))
    data = [data[i * size:(i + 1) * size] for i in range(processes)]
    data = pool.map(merge_sort, data)
    while len(data) > 1:
        extra = data.pop() if len(data) % 2 == 1 else None
        data = [(data[i], data[i + 1]) for i in range(0, len(data), 2)]
        data = pool.map(merge, data) + ([extra] if extra else [])
    return data[0]

# Execute Merge Sort

In [9]:
# Sequential Merge Sort
Merge_Sequential = {}
try:
  print('Sequential Merge Sort HundredThousand Begin')
  data = HundredThousand.copy()
  start = datetime.now()
  mergeSort(data)
  end = datetime.now()
  time = end - start
  del data
  Merge_Sequential['HundredThousand'] = time.total_seconds()
  print('Sequential Merge Sort HundredThousand Done')
except:
  Merge_Sequential['HundredThousand'] = 0
  print('Sequential Merge Sort HundredThousand Failed')
try:
  print('Sequential Merge Sort OneMillion Begin')
  data = OneMillion.copy()
  start = datetime.now()
  mergeSort(data)
  end = datetime.now()
  time = end - start
  del data
  Merge_Sequential['OneMillion'] = time.total_seconds()
  print('Sequential Merge Sort OneMillion Done')
except:
  Merge_Sequential['OneMillion'] = 0
  print('Sequential Merge Sort OneMillion Failed')
try:
  print('Sequential Merge Sort TenMillion Begin')
  data = TenMillion.copy()
  start = datetime.now()
  mergeSort(data)
  end = datetime.now()
  time = end - start
  del data
  Merge_Sequential['TenMillion'] = time.total_seconds()
  print('Sequential Merge Sort TenMillion Done')
except:
  Merge_Sequential['TenMillion'] = 0
  print('Sequential Merge Sort TenMillion Failed')
try:
  print('Sequential Merge Sort FiftyMillion Begin')
  data = FiftyMillion.copy()
  start = datetime.now()
  mergeSort(data)
  end = datetime.now()
  time = end - start
  del data
  Merge_Sequential['FiftyMillion'] = time.total_seconds()
  print('Sequential Merge Sort FiftyMillion Done')
except:
  Merge_Sequential['FiftyMillion'] = 0
  print('Sequential Merge Sort FiftyMillion Failed')
try:
  print('Sequential Merge Sort HundredMillion Begin')
  data = HundredMillion.copy()
  start = datetime.now()
  mergeSort(data)
  end = datetime.now()
  time = end - start
  del data
  Merge_Sequential['HundredMillion'] = time.total_seconds()
  print('Sequential Merge Sort HundredMillion Done')
except:
  Merge_Sequential['HundredMillion'] = 0
print('Sequential Merge Sort : ',Merge_Sequential,'\n')

Sequential Merge Sort HundredThousand Begin
Sequential Merge Sort HundredThousand Done
Sequential Merge Sort OneMillion Begin
Sequential Merge Sort OneMillion Done
Sequential Merge Sort TenMillion Begin
Sequential Merge Sort TenMillion Done
Sequential Merge Sort FiftyMillion Begin
Sequential Merge Sort FiftyMillion Done
Sequential Merge Sort HundredMillion Begin
Sequential Merge Sort HundredMillion Done
Sequential Merge Sort :  {'HundredThousand': 0.583653, 'OneMillion': 7.177781, 'TenMillion': 88.373841, 'FiftyMillion': 502.488433, 'HundredMillion': 1058.762693} 



In [10]:
# Parallel Merge Sort
Merge_Parallel = {}
try:
  print('Parallel Merge Sort HundredThousand Begin')
  data = HundredThousand.copy()
  start = datetime.now()
  merge_sort(data)
  end = datetime.now()
  time = end - start
  del data
  Merge_Parallel['HundredThousand'] = time.total_seconds()
  print('Parallel Merge Sort HundredThousand Done')
except:
  Merge_Parallel['HundredThousand'] = 0
  print('Parallel Merge Sort HundredThousand Failed')
try:
  print('Parallel Merge Sort OneMillion Begin')
  data = OneMillion.copy()
  start = datetime.now()
  merge_sort(data)
  end = datetime.now()
  time = end - start
  del data
  Merge_Parallel['OneMillion'] = time.total_seconds()
  print('Parallel Merge Sort OneMillion Done')
except:
  Merge_Parallel['OneMillion'] = 0
  print('Parallel Merge Sort OneMillion Failed')
try:
  print('Parallel Merge Sort TenMillion Begin')
  data = TenMillion.copy()
  start = datetime.now()
  merge_sort(data)
  end = datetime.now()
  time = end - start
  del data
  Merge_Parallel['TenMillion'] = time.total_seconds()
  print('Parallel Merge Sort TenMillion Done')
except:
  Merge_Parallel['TenMillion'] = 0
  print('Parallel Merge Sort TenMillion Failed')
try:
  print('Parallel Merge Sort FiftyMillion Begin')
  data = FiftyMillion.copy()
  start = datetime.now()
  merge_sort(data)
  end = datetime.now()
  time = end - start
  del data
  Merge_Parallel['FiftyMillion'] = time.total_seconds()
  print('Parallel Merge Sort FiftyMillion Done')
except:
  Merge_Parallel['FiftyMillion'] = 0
  print('Parallel Merge Sort FiftyMillion Failed')
try:
  print('Parallel Merge Sort HundredMillion Begin')
  data = HundredMillion.copy()
  start = datetime.now()
  merge_sort(data)
  end = datetime.now()
  time = end - start
  del data
  Merge_Parallel['HundredMillion'] = time.total_seconds()
  print('Parallel Merge Sort HundredMillion Done')
except:
  Merge_Parallel['HundredMillion'] = 0
  print('Parallel Merge Sort HundredMillion Failed')
print('Parallel Merge Sort : ',Merge_Parallel,'\n')

Parallel Merge Sort HundredThousand Begin
Parallel Merge Sort HundredThousand Done
Parallel Merge Sort OneMillion Begin
Parallel Merge Sort OneMillion Done
Parallel Merge Sort TenMillion Begin
Parallel Merge Sort TenMillion Done
Parallel Merge Sort FiftyMillion Begin
Parallel Merge Sort FiftyMillion Done
Parallel Merge Sort HundredMillion Begin
Parallel Merge Sort HundredMillion Done
Parallel Merge Sort :  {'HundredThousand': 3.518121, 'OneMillion': 5.150112, 'TenMillion': 63.922264, 'FiftyMillion': 364.765004, 'HundredMillion': 773.553118} 



# Execute Quick Sort


In [11]:
# Sequential Quick Sort
Quick_Sequential = {}
try:
  print('Sequential Quick Sort HundredThousand Begin')
  data = HundredThousand.copy()
  start = datetime.now()
  quickSort(data,False,False)
  end = datetime.now()
  time = end - start
  del data
  Quick_Sequential['HundredThousand'] = time.total_seconds()
  print('Sequential Quick Sort HundredThousand Done')
except:
  Quick_Sequential['HundredThousand'] = 0
  print('Sequential Quick Sort HundredThousand Failed')
try:
  print('Sequential Quick Sort OneMillion Begin')
  data = OneMillion.copy()
  start = datetime.now()
  quickSort(data,False,False)
  end = datetime.now()
  time = end - start
  del data
  Quick_Sequential['OneMillion'] = time.total_seconds()
  print('Sequential Quick Sort OneMillion Done')
except:
  Quick_Sequential['OneMillion'] = 0
  print('Sequential Quick Sort OneMillion Failed')
try:
  print('Sequential Quick Sort TenMillion Begin')
  data = TenMillion.copy()
  start = datetime.now()
  quickSort(data,False,False)
  end = datetime.now()
  time = end - start
  del data
  Quick_Sequential['TenMillion'] = time.total_seconds()
  print('Sequential Quick Sort TenMillion Done')
except:
  Quick_Sequential['TenMillion'] = 0
  print('Sequential Quick Sort TenMillion Failed')
try:
  print('Sequential Quick Sort FiftyMillion Begin')
  data = FiftyMillion.copy()
  start = datetime.now()
  quickSort(data,False,False)
  end = datetime.now()
  time = end - start
  del data
  Quick_Sequential['FiftyMillion'] = time.total_seconds()
  print('Sequential Quick Sort FiftyMillion Done')
except:
  Quick_Sequential['FiftyMillion'] = 0
  print('Sequential Quick Sort FiftyMillion Failed')
try:
  print('Sequential Quick Sort HundredMillion Begin')
  data = HundredMillion.copy()
  start = datetime.now()
  quickSort(data,False,False)
  end = datetime.now()
  time = end - start
  del data
  Quick_Sequential['HundredMillion'] = time.total_seconds()
  print('Sequential Quick Sort HundredMillion Done')
except:
  Quick_Sequential['HundredMillion'] = 0
  print('Sequential Quick Sort HundredMillion Failed')
print('Sequential Quick Sort : ',Quick_Sequential,'\n')

Sequential Quick Sort HundredThousand Begin
Sequential Quick Sort HundredThousand Done
Sequential Quick Sort OneMillion Begin
Sequential Quick Sort OneMillion Done
Sequential Quick Sort TenMillion Begin
Sequential Quick Sort TenMillion Done
Sequential Quick Sort FiftyMillion Begin
Sequential Quick Sort FiftyMillion Done
Sequential Quick Sort HundredMillion Begin
Sequential Quick Sort HundredMillion Done
Sequential Quick Sort :  {'HundredThousand': 0.86409, 'OneMillion': 5.759202, 'TenMillion': 73.573393, 'FiftyMillion': 431.178253, 'HundredMillion': 908.592036} 



In [12]:
# Parallel Quick Sort
Quick_Parallel = {}
try:
  print('Parallel Quick Sort HundredThousand Begin')
  data = HundredThousand.copy()
  start = datetime.now()
  quickSort(data,True,True)
  end = datetime.now()
  time = end - start
  del data
  Quick_Parallel['HundredThousand'] = time.total_seconds()
  print('Parallel Quick Sort HundredThousand Done')
except:
  Quick_Parallel['HundredThousand'] = 0
  print('Parallel Quick Sort HundredThousand Failed')
try:
  print('Parallel Quick Sort OneMillion Begin')
  data = OneMillion.copy()
  start = datetime.now()
  quickSort(data,True,True)
  end = datetime.now()
  time = end - start
  del data
  Quick_Parallel['OneMillion'] = time.total_seconds()
  print('Parallel Quick Sort OneMillion Done')
except:
  Quick_Parallel['OneMillion'] = 0
  print('Parallel Quick Sort OneMillion Failed')
try:
  print('Parallel Quick Sort TenMillion Begin')
  data = TenMillion.copy()
  start = datetime.now()
  quickSort(data,True,True)
  end = datetime.now()
  time = end - start
  del data
  Quick_Parallel['TenMillion'] = time.total_seconds()
  print('Parallel Quick Sort TenMillion Done')
except:
  Quick_Parallel['TenMillion'] = 0
  print('Parallel Quick Sort TenMillion Failed')
try:
  print('Parallel Quick Sort FiftyMillion Begin')
  data = FiftyMillion.copy()
  start = datetime.now()
  quickSort(data,True,True)
  end = datetime.now()
  time = end - start
  del data
  Quick_Parallel['FiftyMillion'] = time.total_seconds()
  print('Parallel Quick Sort FiftyMillion Done')
except:
  Quick_Parallel['FiftyMillion'] = 0
  print('Parallel Quick Sort FiftyMillion Failed')
try:
  print('Parallel Quick Sort HundredMillion Begin')
  data = HundredMillion.copy()
  start = datetime.now()
  quickSort(data,True,True)
  end = datetime.now()
  time = end - start
  del data
  Quick_Parallel['HundredMillion'] = time.total_seconds()
  print('Parallel Quick Sort HundredMillion Done')
except:
  Quick_Parallel['HundredMillion'] = 0
  print('Parallel Quick Sort HundredMillion Failed')
print('Parallel Quick Sort : ',Quick_Parallel,'\n')

Parallel Quick Sort HundredThousand Begin
Parallel Quick Sort HundredThousand Done
Parallel Quick Sort OneMillion Begin
Parallel Quick Sort OneMillion Done
Parallel Quick Sort TenMillion Begin
Parallel Quick Sort TenMillion Done
Parallel Quick Sort FiftyMillion Begin
Parallel Quick Sort FiftyMillion Done
Parallel Quick Sort HundredMillion Begin
Parallel Quick Sort HundredMillion Done
Parallel Quick Sort :  {'HundredThousand': 8.379572, 'OneMillion': 5.792294, 'TenMillion': 78.166659, 'FiftyMillion': 440.790708, 'HundredMillion': 939.984868} 



# Plotting the Results

In [13]:
print('Plotting and Exporting Begin')

x = ['100K','1M','10M','50M','100M']
y0 = [0 for _ in range(len(x))]

Plotting and Exporting Begin


In [14]:
# Quick Sort Metrics
Quick_Sequential = [Quick_Sequential['HundredThousand'],Quick_Sequential['OneMillion'],Quick_Sequential['TenMillion'],Quick_Sequential['FiftyMillion'],Quick_Sequential['HundredMillion']]
Quick_Parallel = [Quick_Parallel['HundredThousand'],Quick_Parallel['OneMillion'],Quick_Parallel['TenMillion'],Quick_Parallel['FiftyMillion'],Quick_Parallel['HundredMillion']]

# Merge Sort Metrics
Merge_Sequential = [Merge_Sequential['HundredThousand'],Merge_Sequential['OneMillion'],Merge_Sequential['TenMillion'],Merge_Sequential['FiftyMillion'],Merge_Sequential['HundredMillion']]
Merge_Parallel = [Merge_Parallel['HundredThousand'],Merge_Parallel['OneMillion'],Merge_Parallel['TenMillion'],Merge_Parallel['FiftyMillion'],Merge_Parallel['HundredMillion']]

In [15]:
# Plotting Merge Sort Metrics
plt.plot(x, Merge_Sequential, color='blue', label='Sequential Merge Sort')
plt.plot(x, Merge_Parallel, color='green', label='Parallel Merge Sort')
plt.xlabel('Number of Integers to Sort')
plt.ylabel('Time Taken in Seconds')
plt.legend()
plt.title('Execution Time of Merge Sort Algorithms')
plt.savefig('Plots/MergeSort.png', dpi=150)
print('MergeSort.png Exported')
plt.clf()

MergeSort.png Exported


<Figure size 432x288 with 0 Axes>

In [16]:
# Plotting Quick Sort Metrics
plt.plot(x, Quick_Sequential, color='blue', linestyle='dotted', label='Sequential Quick Sort')
plt.plot(x, Quick_Parallel, color='green', linestyle='dotted', label='Parallel Quick Sort')
plt.xlabel('Number of Integers to Sort')
plt.ylabel('Time Taken in Seconds')
plt.legend()
plt.title('Execution Time of Quick Sort Algorithms')
plt.savefig('Plots/QuickSort.png', dpi=150)
print('QuickSort.png Exported')
plt.clf()

QuickSort.png Exported


<Figure size 432x288 with 0 Axes>

In [17]:
# Plotting All Metrics In One Graph
plt.plot(x, Quick_Sequential, color='blue', linestyle='dotted', label='Sequential Quick Sort')
plt.plot(x, Quick_Parallel, color='green', linestyle='dotted', label='Parallel Quick Sort')
plt.plot(x, Merge_Sequential, color='blue', label='Sequential Merge Sort')
plt.plot(x, Merge_Parallel, color='green', label='Parallel Merge Sort')
plt.xlabel('Number of Integers to Sort')
plt.ylabel('Time Taken in Seconds')
plt.legend()
plt.title('Execution Time of All Sorting Algorithms')
plt.savefig('Plots/Sort.png', dpi=150)
print('Sort.png Exported')
plt.clf()

Sort.png Exported


<Figure size 432x288 with 0 Axes>